# Binary Semantic Segmentation using TAO UNET

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://d29g4g2dyqv443.cloudfront.net/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png" width="1080"> 

## Learning Objectives
In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Take a pretrained resnet18 model and train a ResNet-18 UNet model on the ISBI dataset
* Run Inference on the trained model and visualize the inferences
* Export the trained model to a .onnx file for deployment to DeepStream
* Run inference on the exported .onnx model to verify deployment using TensorRT

At the end of this notebook, you will have generated a trained and optimized `UNet` model trained
on the ISBI dataset. You may then deploy this model via [Triton](https://github.com/NVIDIA-AI-IOT/tao-toolkit-triton-apps)
or [DeepStream](https://developer.nvidia.com/deepstream-sdk).

### Table of Contents

This notebook shows an example use case of UNet Binary Semantic Segmentation using Train Adapt Optimize (TAO) Toolkit.

0. [Set up env variables](#head-0)
1. [Installing the TAO launcher](#head-1) <br>
2. [Prepare dataset and pre-trained model](#head-2) <br>
    1. [Verify downloaded dataset](#head-2-1)
    1. [Prepare ISBI data from TIF](#head-2-2)
    2.[Visualize the Groundtruth Masks](#head-2-3)
    3. [Download pre-trained model](#head-2-4)
3. [Provide training specification](#head-3)
4. [Run TAO training](#head-4)
5. [Evaluate trained models](#head-5)
6. [Visualizing Inferences](#head-6)
7. [Model Export](#head-7)
8. [Verify the deployed model](#head-8)
9. [QAT workflow](#head-9)
   1. [Convert pruned model to QAT and retrain](#head-9-1)
   2. [Evaluate QAT converted model](#head-9-2)
   3. [Export QAT trained model to int8](#head-9-3)
   4. [Evaluate a QAT trained model using the exported TensorRT engine](#head-9-4)
   5. [Inference using QAT engine](#head-9-5)

### Sample prediction of UNet
<img align="center" src="https://github.com/vpraveen-nv/model_card_images/blob/main/cv/notebook/unet/sample.jpg?raw=true" width="300" height="300">

## 0. Set up env variables <a class="anchor" id="head-0"></a>

The following notebook requires the user to set an env variable called the `$LOCAL_PROJECT_DIR` as the path to the users workspace. Please note that the dataset to run this notebook is expected to reside in the `$LOCAL_PROJECT_DIR/data/isbi`, while the TAO experiment generated collaterals will be output to `$LOCAL_PROJECT_DIR/unet`. More information on how to set up the dataset and the supported steps in the TAO workflow are provided in the subsequent cells.

*Note: Please make sure to remove any stray artifacts/files from the `$USER_EXPERIMENT_DIR` or `$DATA_DOWNLOAD_DIR` paths as mentioned below, that may have been generated from previous experiments. Having checkpoint files etc may interfere with creating a training graph for a new experiment.*

*Note: This notebook currently is by default set up to run training using 1 GPU. To use more GPU's please update the env variable `$NUM_GPUS` accordingly*


In [ ]:
# Setting up env variables for cleaner command line commands.
import os

%set_env GPU_INDEX="0"
%env NUM_GPUS=1
%set_env USER_EXPERIMENT_DIR=/workspace/tao-experiments/unet
%set_env DATA_DOWNLOAD_DIR=/workspace/tao-experiments/data

# Set this path if you don't run the notebook from the samples directory.
# %env NOTEBOOK_ROOT=~/tao-samples/unet

# Please define this local project directory that needs to be mapped to the TAO docker session.
# The dataset expected to be present in $LOCAL_PROJECT_DIR/data, while the results for the steps
# in this notebook will be stored at $LOCAL_PROJECT_DIR/unet
# !PLEASE MAKE SURE TO UPDATE THIS PATH!.
os.environ["LOCAL_PROJECT_DIR"] = FIXME

# !PLEASE MAKE SURE TO UPDATE THIS PATH!.
# Point to the 'deps' folder in samples from where you are launching notebook inside unet folder.
%env PROJECT_DIR=/workspace/iva/ngc-collaterals/cv/samples

os.environ["LOCAL_DATA_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "data"
)
os.environ["LOCAL_EXPERIMENT_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "unet"
)

# The sample spec files are present in the same path as the downloaded samples.
os.environ["LOCAL_SPECS_DIR"] = os.path.join(
    os.getenv("NOTEBOOK_ROOT", os.getcwd()),
    "specs"
)
%set_env SPECS_DIR=/workspace/tao-experiments/unet/specs

! ls -l $LOCAL_DATA_DIR
!ls -rlt $LOCAL_SPECS_DIR

The cell below maps the project directory on your local host to a workspace directory in the TAO docker instance, so that the data and the results are mapped from in and out of the docker. For more information please refer to the [launcher instance](https://docs.nvidia.com/tao/tao-toolkit/text/tao_launcher.html) in the user guide.

When running this cell on AWS, update the drive_map entry with the dictionary defined below, so that you don't have permission issues when writing data into folders created by the TAO docker.

```json
drive_map = {
    "Mounts": [
            # Mapping the data directory
            {
                "source": os.environ["LOCAL_PROJECT_DIR"],
                "destination": "/workspace/tao-experiments"
            },
            # Mapping the specs directory.
            {
                "source": os.environ["LOCAL_SPECS_DIR"],
                "destination": os.environ["SPECS_DIR"]
            },
        ],
    "DockerOptions": {
        "user": "{}:{}".format(os.getuid(), os.getgid())
    }
}
```

In [ ]:
# Mapping up the local directories to the TAO docker.
import json
mounts_file = os.path.expanduser("~/.tao_mounts.json")

# Define the dictionary with the mapped drives
drive_map = {
    "Mounts": [
        # Mapping the data directory
        {
            "source": os.environ["LOCAL_PROJECT_DIR"],
            "destination": "/workspace/tao-experiments"
        },
        # Mapping the specs directory.
        {
            "source": os.environ["LOCAL_SPECS_DIR"],
            "destination": os.environ["SPECS_DIR"]
        },
    ]
}

# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(drive_map, mfile, indent=4)

In [ ]:
!cat ~/.tao_mounts.json

In [ ]:
# Install requirement
!pip3 install Cython==0.29.36
!pip3 install -r $PROJECT_DIR/deps/requirements-pip.txt

## 1. Installing the TAO launcher <a class="anchor" id="head-1"></a>
The TAO launcher is a python package distributed as a python wheel listed in PyPI. You may install the launcher by executing the following cell.

Please note that TAO Toolkit recommends users to run the TAO launcher in a virtual env with python 3.6.9. You may follow the instruction in this [page](https://virtualenvwrapper.readthedocs.io/en/latest/install.html) to set up a python virtual env using the `virtualenv` and `virtualenvwrapper` packages. Once you have setup virtualenvwrapper, please set the version of python to be used in the virtual env by using the `VIRTUALENVWRAPPER_PYTHON` variable. You may do so by running

```sh
export VIRTUALENVWRAPPER_PYTHON=/path/to/bin/python3.x
```
where x >= 6 and <= 8

We recommend performing this step first and then launching the notebook from the virtual environment. In addition to installing TAO python package, please make sure of the following software requirements:
* python >=3.7, <=3.10.x
* docker-ce > 19.03.5
* docker-API 1.40
* nvidia-container-toolkit > 1.3.0-1
* nvidia-container-runtime > 3.4.0-1
* nvidia-docker2 > 2.5.0-1
* nvidia-driver > 455+

Once you have installed the pre-requisites, please log in to the docker registry nvcr.io by following the command below

```sh
docker login nvcr.io
```

You will be triggered to enter a username and password. The username is `$oauthtoken` and the password is the API key generated from `ngc.nvidia.com`. Please follow the instructions in the [NGC setup guide](https://docs.nvidia.com/ngc/ngc-overview/index.html#generating-api-key) to generate your own API key.

In [ ]:
# SKIP this step IF you have already installed the TAO launcher.
!pip3 install nvidia-tao

In [ ]:
# View the versions of the TAO launcher
!tao info

## 2. Prepare dataset and pre-trained model <a class="anchor" id="head-1"></a>

We will be using the `ISBI Challenge: Segmentation of neuronal structures in EM stacks` dataset for the binary segmentation tutorial. Please access the open source repo here https://github.com/alexklibisz/isbi-2012/tree/master/data to download the data. The data is in .tif format. Copy the train-labels.tif, train-volume.tif, test-volume.tif files to `$LOCAL_DATA_DIR/isbi`. 

### A. Verify downloaded dataset <a class="anchor" id="head-1-1"></a>

In [ ]:
# Check the .tiff files are present
!mkdir -p $LOCAL_DATA_DIR
!ls -l $LOCAL_DATA_DIR
!if [ ! -f $LOCAL_DATA_DIR/isbi/test-volume.tif ]; then echo 'test-volume.tif file not found, please download.'; else echo 'Found test-volume.tif file.';fi
!if [ ! -f $LOCAL_DATA_DIR/isbi/test-volume.tif ]; then echo 'train-labels file not found, please download.'; else echo 'Found train-labels.tif file.';fi
!if [ ! -f $LOCAL_DATA_DIR/isbi/test-volume.tif ]; then echo 'train-volume.tif file not found, please download.'; else echo 'Found train-volume.tif file.';fi

### B.  Prepare ISBI data from TIF <a class="anchor" id="head-1-2"></a>

Prepare the images and masks from the .tif files by running the following script.

In [ ]:
!bash prepare_data.sh $LOCAL_DATA_DIR/isbi

In [ ]:
!if [ -f $DATA_DOWNLOAD_DIR/isbi/images/train/image_0.png ]; then echo 'Data prepared successfully !'; else echo 'Please verify Data Preparation.';fi
# Checking the files
!ls -l $LOCAL_DATA_DIR/isbi
!ls -l $LOCAL_DATA_DIR/isbi/images
!ls -l $LOCAL_DATA_DIR/isbi/masks
!ls -l $LOCAL_DATA_DIR/isbi/images/train

### C.  Visualize the Groundtruth Masks <a class="anchor" id="head-1-3"></a>

In [ ]:
# Run this cell to visualize the training data and the masks

!python vis_annotation_isbi.py -i $LOCAL_DATA_DIR/isbi/images/train -m $LOCAL_DATA_DIR/isbi/masks/train -o $LOCAL_EXPERIMENT_DIR/isbi_experiment_unpruned/vis_gt --num_classes 2 --num_images 10

In [ ]:
# Simple grid visualizer
%matplotlib inline

import matplotlib.pyplot as plt
import os
from math import ceil
valid_image_ext = ['.jpg', '.png']

def visualize_images(image_dir, result_dir, num_cols=4, num_images=10):
    output_path = os.path.join(os.environ['LOCAL_EXPERIMENT_DIR'], result_dir, image_dir)
    num_rows = int(ceil(float(num_images) / float(num_cols)))
    f, axarr = plt.subplots(num_rows, num_cols, figsize=[80,30])
    f.tight_layout()
    a = [os.path.join(output_path, image) for image in os.listdir(output_path) 
         if os.path.splitext(image)[1].lower() in valid_image_ext]
    for idx, img_path in enumerate(a[:num_images]):
        col_id = idx % num_cols
        row_id = idx // num_cols
        img = plt.imread(img_path)
        axarr[row_id, col_id].imshow(img) 

In [ ]:
# Visualizing the sample images.
OUTPUT_PATH = 'vis_gt' # relative path from $USER_EXPERIMENT_DIR.
COLS = 2 # number of columns in the visualizer grid.
IMAGES = 4 # number of images to visualize.

visualize_images(OUTPUT_PATH, "isbi_experiment_unpruned", num_cols=COLS, num_images=IMAGES)

### D. Download pre-trained model <a class="anchor" id="head-1-4"></a>

We will use NGC CLI to get the pre-trained models. For more details, go to ngc.nvidia.com and click the SETUP on the navigation bar. 

*Note: When using vanilla_unet as arch for binary segmentation, pre-trained model section can be skipped. Pre-trained weights are available only for Resnet/ VGG templates*

In [ ]:
# Installing NGC CLI on the local machine.
## Download and install
%env CLI=ngccli_cat_linux.zip
!mkdir -p $LOCAL_PROJECT_DIR/ngccli

# Remove any previously existing CLI installations
!rm -rf $LOCAL_PROJECT_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli
!unzip -u "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
!rm $LOCAL_PROJECT_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli/ngc-cli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""), os.getenv("PATH", ""))

In [ ]:
!ngc registry model list nvidia/tao/pretrained_semantic_segmentation:*

In [ ]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/pretrained_resnet18/

In [ ]:
# Pull pretrained model from NGC
!ngc registry model download-version nvidia/tao/pretrained_semantic_segmentation:resnet18 --dest $LOCAL_EXPERIMENT_DIR/pretrained_resnet18

In [ ]:
print("Check that model is downloaded into dir.")
!ls -l $LOCAL_EXPERIMENT_DIR/pretrained_resnet18/pretrained_semantic_segmentation_vresnet18

## 3. Provide ISBI training specification <a class="anchor" id="head-3"></a>

* Images and Masks path
    * In order to use the newly generated images, masks folder update the dataset_config parameter in the spec file at `$SPECS_DIR/unet_train_resnet_unet_isbi.txt` 
    * Update the train, val images and masks paths. The test only requires the images path. 
* Pre-trained models
* Augmentation parameters for on the fly data augmentation
* Other training (hyper-)parameters such as batch size, number of epochs, learning rate etc.


In [ ]:
!cat $LOCAL_SPECS_DIR/unet_train_resnet_unet_isbi.txt

## 4. Run TAO training <a class="anchor" id="head-4"></a>
* Provide the sample spec file and the output directory location for models
* WARNING: training will take several hours or one day to complete

In [ ]:
print("For multi-GPU, change --gpus based on your machine.")
!tao model unet train --gpus $NUM_GPUS \
                      --gpu_index $GPU_INDEX \
                      -e $SPECS_DIR/unet_train_resnet_unet_isbi.txt \
                      -r $USER_EXPERIMENT_DIR/isbi_experiment_unpruned \
                      -m $USER_EXPERIMENT_DIR/pretrained_resnet18/pretrained_semantic_segmentation_vresnet18/resnet_18.hdf5 \
                      -n model_isbi

Unet supports restarting from checkpoint. In case, the training job is killed prematurely, you may resume training from the closest checkpoint by simply re-running the same command line. Please do make sure to use the same number of GPUs when restarting the training.

In [ ]:
print('Model for every epoch at checkpoint_interval mentioned in the spec file:')
print('---------------------')
!ls -ltrh $LOCAL_EXPERIMENT_DIR/isbi_experiment_unpruned/
!ls -ltrh $LOCAL_EXPERIMENT_DIR/isbi_experiment_unpruned/weights

## 5. Evaluate trained models <a class="anchor" id="head-5"></a>

The last step model saved in the `$USER_EXPERIMENT_DIR/isbi_experiment_unpruned/weights` dir is used for evaluation/ inference/ export. The evaluation also creates `$LOCAL_EXPERIMENT_DIR/isbi_experiment_unpruned/results_tlt.json`

In [ ]:
!tao model unet evaluate --gpu_index $GPU_INDEX -e $SPECS_DIR/unet_train_resnet_unet_isbi.txt \
                 -m $USER_EXPERIMENT_DIR/isbi_experiment_unpruned/weights/model_isbi.tlt \
                 -o $USER_EXPERIMENT_DIR/isbi_experiment_unpruned/ 

In [ ]:
!cat $LOCAL_EXPERIMENT_DIR/isbi_experiment_unpruned/results_tlt.json

## 6. Visualizing Inferences <a class="anchor" id="head-6"></a>
In this section, we run the UNet inference tool to generate inferences on the trained models and print the results. 

The following cell will run inference for segmentation and visualize masks for the images in test. The resulting visualized images will be saved in the `vis_overlay_tlt` folder and label PNG masks in `mask_labels_tlt` in the path provided to `-o` argument.


In [ ]:
!tao model unet inference --gpu_index $GPU_INDEX -e $SPECS_DIR/unet_train_resnet_unet_isbi.txt \
                 -m $USER_EXPERIMENT_DIR/isbi_experiment_unpruned/weights/model_isbi.tlt \
                 -o $USER_EXPERIMENT_DIR/isbi_experiment_unpruned/ 

In [ ]:
# Visualizing the sample images.
OUTPUT_PATH = 'vis_overlay_tlt' # relative path from $USER_EXPERIMENT_DIR.
COLS = 2 # number of columns in the visualizer grid.
IMAGES = 4 # number of images to visualize.

visualize_images(OUTPUT_PATH, "isbi_experiment_unpruned", num_cols=COLS, num_images=IMAGES)

## 6. Prune the trained model <a class="anchor" id="head-6"></a>
* Specify pre-trained model
* Equalization criterion (`Applicable for resnets`)
* Threshold for pruning.
* Output directory to store the model

*Usually, you just need to adjust `-pth` (threshold) for accuracy and model size trade off. Higher `pth` gives you smaller model (and thus higher inference speed) but worse accuracy. The threshold to use is dependent on the dataset. A pth value `5.2e-6` is just a start point. If the retrain accuracy is good, you can increase this value to get smaller models. Otherwise, lower this value to get better accuracy.*

* For some internal studies, we have noticed that a pth value of 0.1 is a good starting point for unet models trained on larger datasets. A larger regularization value in the first round of training will result in smaller models while pruning. Hence regularization while training and pth are hyper-parameters that needs to be tuned.*

In [ ]:
# Create an output directory if it doesn't exist.
!mkdir -p $LOCAL_EXPERIMENT_DIR/isbi_experiment_pruned

In [ ]:
!tao model unet prune \
                  -e $SPECS_DIR/unet_train_resnet_unet_isbi.txt \
                  -m $USER_EXPERIMENT_DIR/isbi_experiment_unpruned/weights/model_isbi.tlt \
                  -o $USER_EXPERIMENT_DIR/isbi_experiment_pruned/model_isbi_pruned.tlt \
                  -eq union \
                  -pth 0.6 

In [ ]:
!ls -rlt $LOCAL_EXPERIMENT_DIR/isbi_experiment_pruned/

## 7. Retrain the pruned model <a class="anchor" id="head-7"></a>
* Model needs to be re-trained to bring back accuracy after pruning
* Specify re-training specification with pretrained weights as pruned model.

*Note: For retraining, please set the `load_graph` option to `true` in the model_config to load the pruned model graph. Also, if after retraining, the model shows some decrease in MIOU, it could be that the originally trained model was pruned a little too much. Please try reducing the pruning threshold (thereby reducing the pruning ratio) and use the new model to retrain.*

*Note: Ensure to provide a different folder for saving results of retraining from the folder where pruned model is saved.

In [ ]:
# Printing the retrain experiment file. 
# Note: We have updated the experiment file to include the 
# newly pruned model as a pretrained weights and, the
# load_graph option is set to true 
!cat $LOCAL_SPECS_DIR/unet_train_resnet_unet_isbi_retrain.txt

In [ ]:
# Retraining using the pruned model as pretrained weights 
!tao model unet train --gpus $NUM_GPUS \
                      --gpu_index $GPU_INDEX \
                      -e $SPECS_DIR/unet_train_resnet_unet_isbi_retrain.txt \
                      -r $USER_EXPERIMENT_DIR/isbi_experiment_retrain \
                      -m $USER_EXPERIMENT_DIR/isbi_experiment_pruned/model_isbi_pruned.tlt \
                      -n model_isbi_retrained 

In [ ]:
# Listing the newly retrained model.
!ls -rlt $LOCAL_EXPERIMENT_DIR/isbi_experiment_retrain/weights

## 8. Evaluate the retrained model <a class="anchor" id="head-8"></a>

This section evaluates the pruned and retrained model, using the `evaluate` command.

In [ ]:
!tao model unet evaluate --gpu_index $GPU_INDEX -e $SPECS_DIR/unet_train_resnet_unet_isbi_retrain.txt \
                 -m $USER_EXPERIMENT_DIR/isbi_experiment_retrain/weights/model_isbi_retrained.tlt \
                 -o $USER_EXPERIMENT_DIR/isbi_experiment_retrain/ 

## 7. Model Export <a class="anchor" id="head-7"></a>

In [ ]:
# tao <task> export will fail if .onnx already exists. So we clear the export folder before tao <task> export
!rm -rf $LOCAL_EXPERIMENT_DIR/export
# Generate .onnx file using tao container
!mkdir -p $LOCAL_EXPERIMENT_DIR/export 

!tao model unet export --gpu_index $GPU_INDEX -m $USER_EXPERIMENT_DIR/isbi_experiment_retrain/weights/model_isbi_retrained.tlt \
               -e $SPECS_DIR/unet_train_resnet_unet_isbi_retrain.txt \
               --gen_ds_config

In [ ]:
# Check if onnx model is correctly saved.
!ls -l $LOCAL_EXPERIMENT_DIR/isbi_experiment_retrain/weights/

Using the `tao deploy` container, you can generate a TensorRT engine and verify the correctness of the generated through evaluate and inference. 

The `tao deploy` produces optimized tensorrt engines for the platform that it resides on. Therefore, to get maximum performance, please run `tao deploy` command which will instantiate a deploy container, with the exported `.onnx` file on your target device. The `tao deploy` container only works for x86, with discrete NVIDIA GPU's. 

If you choose to integrate your model into deepstream directly, please refer to [deepstream dev guide](https://docs.nvidia.com/tao/tao-toolkit/text/ds_tao/deepstream_tao_integration.html) for more details.

In [ ]:
# Convert to TensorRT engine(FP32).
!tao deploy unet gen_trt_engine --gpu_index $GPU_INDEX \
                                -m $USER_EXPERIMENT_DIR/isbi_experiment_retrain/weights/model_isbi_retrained.onnx \
                                -e $SPECS_DIR/unet_train_resnet_unet_isbi_retrain.txt \
                                -r $USER_EXPERIMENT_DIR/export \
                                --data_type fp32 \
                                --engine_file $USER_EXPERIMENT_DIR/export/trtfp32.isbi.retrained.engine \
                                --max_batch_size 3

In [ ]:
# Verify the tensorrt engine accuracy on the validation dataset through tao deploy. This generates results text file 
# results.json in the directory mentioned to the -r argument 

!tao deploy unet evaluate --gpu_index $GPU_INDEX -e $SPECS_DIR/unet_train_resnet_unet_isbi_retrain.txt \
                          -m $USER_EXPERIMENT_DIR/export/trtfp32.isbi.retrained.engine \
                          -r $USER_EXPERIMENT_DIR/isbi_experiment_retrain/

In [ ]:
!cat $LOCAL_EXPERIMENT_DIR/isbi_experiment_retrain/results.json

In [ ]:
# Convert to TensorRT engine(INT8).
!mkdir -p $LOCAL_EXPERIMENT_DIR/export 
!tao deploy unet gen_trt_engine --gpu_index $GPU_INDEX -m $USER_EXPERIMENT_DIR/isbi_experiment_retrain/weights/model_isbi_retrained.onnx \
                                -e $SPECS_DIR/unet_train_resnet_unet_isbi_retrain.txt \
                                -r $USER_EXPERIMENT_DIR/export \
                                --data_type int8 \
                                --engine_file $USER_EXPERIMENT_DIR/export/int8.isbi.retrained.engine \
                                --data_type int8 \
                                --cal_data_file $USER_EXPERIMENT_DIR/export/isbi_cal_data_file.txt \
                                --cal_cache_file $USER_EXPERIMENT_DIR/export/isbi_cal.bin \
                                --cal_image_dir $DATA_DOWNLOAD_DIR/isbi/images/val \
                                --max_batch_size 3 \
                                --batch_size 1

In [ ]:
print('Exported model:')
print('------------')
!ls -lh $LOCAL_EXPERIMENT_DIR/export

## 8. Verify the deployed model <a class="anchor" id="head-8"></a>

Verify the converted engine. The resulting TRT inference images will be saved in `vis_overlay` folder and PNG masks in `mask_labels`  in the path provided to `-o` argument.

*Note: The TensorRT evaluation and inference must be run with a batch size smaller than `max_batch_size` that was used while generating engine.*

In [ ]:
# Running inference on test dataset through tao deploy
!tao deploy unet inference --gpu_index $GPU_INDEX -e $SPECS_DIR/unet_train_resnet_unet_isbi_retrain.txt \
                           -m $USER_EXPERIMENT_DIR/export/int8.isbi.retrained.engine \
                           -r $USER_EXPERIMENT_DIR/isbi_experiment_retrain/

In [ ]:
# Running inference on test dataset through tao deploy
!tao deploy unet evaluate --gpu_index $GPU_INDEX -e $SPECS_DIR/unet_train_resnet_unet_isbi_retrain.txt \
                          -m $USER_EXPERIMENT_DIR/export/int8.isbi.retrained.engine \
                          -r $USER_EXPERIMENT_DIR/isbi_experiment_retrain/

In [ ]:
# Visualizing the sample images.
OUTPUT_PATH = 'vis_overlay' # relative path from $USER_EXPERIMENT_DIR.
COLS = 2 # number of columns in the visualizer grid.
IMAGES = 4 # number of images to visualize.

visualize_images(OUTPUT_PATH, "isbi_experiment_retrain", num_cols=COLS, num_images=IMAGES)

## 10. QAT workflow <a class="anchor" id="head-12"></a>
This section delves into the newly enabled Quantization Aware Training feature with UNet. The workflow defined below converts a pruned model from section [5](#head-5) to enable QAT and retrain this model to while accounting the noise introduced due to quantization in the forward pass. 

### A. Convert pruned model to QAT and retrain <a class="anchor" id="head-12-1"></a>
All UNet models, unpruned and pruned models can be converted to QAT models by setting the `enable_qat` parameter in the `training_config` component of the spec file to `true`.

In [ ]:
# Printing the retrain experiment file. 
# Note: We have updated the experiment file to convert the
# pretrained model to qat mode by setting the enable_qat
# parameter.
!cat $LOCAL_SPECS_DIR/unet_train_resnet_unet_isbi_retrain_qat.txt

In [ ]:
!tao model unet train -e $SPECS_DIR/unet_train_resnet_unet_isbi_retrain_qat.txt \
                      -r $USER_EXPERIMENT_DIR/experiment_dir_retrain_qat \
                      -m $USER_EXPERIMENT_DIR/isbi_experiment_pruned/model_isbi_pruned.tlt \
                      -n resnet18_unet_pruned_qat \
                      --gpus $NUM_GPUS \
                      --gpu_index $GPU_INDEX

In [ ]:
!ls -rlt $LOCAL_EXPERIMENT_DIR/experiment_dir_retrain_qat/weights

### B. Evaluate QAT converted model <a class="anchor" id="head-12-2"></a>
This section evaluates a QAT enabled pruned retrained model. The MIOU of this model should be comparable to that of the pruned retrained model without QAT. However, due to quantization, it is possible sometimes to see a drop in the MIOU value for certain datasets.

In [ ]:
!tao model unet evaluate -e $SPECS_DIR/unet_train_resnet_unet_isbi_retrain_qat.txt \
                           -m $USER_EXPERIMENT_DIR/experiment_dir_retrain_qat/weights/resnet18_unet_pruned_qat.tlt \
                           -o $USER_EXPERIMENT_DIR/isbi_experiment_pruned_qat/ 

### C. Export QAT trained model to int8 <a class="anchor" id="head-12-3"></a>
Export a QAT trained model to TensorRT parsable model. This command generates an .onnx file from the trained model and the serializes corresponding int8 scales as a TRT readable calibration cache file.

In [ ]:
# Uncomment the below lines and run for QA to run Int8 export also in the next cell 
!rm -rf $LOCAL_EXPERIMENT_DIR/experiment_dir_retrain_qat/weights/*.onnx

In [ ]:
!tao model unet export --gpu_index $GPU_INDEX -m $USER_EXPERIMENT_DIR/experiment_dir_retrain_qat/weights/resnet18_unet_pruned_qat.tlt \
                       -e $SPECS_DIR/unet_train_resnet_unet_isbi_retrain_qat.txt \
                       --cal_json_file $USER_EXPERIMENT_DIR/export/isbi_cal_qat.json \
                       --gen_ds_config

In [ ]:
# Convert to TensorRT engine
!tao deploy unet gen_trt_engine --gpu_index $GPU_INDEX -m $USER_EXPERIMENT_DIR/experiment_dir_retrain_qat/weights/resnet18_unet_pruned_qat.onnx \
                                -e $SPECS_DIR/unet_train_resnet_unet_isbi_retrain_qat.txt \
                                --engine_file $USER_EXPERIMENT_DIR/export/int8.isbi.retrained_qat.engine \
                                -r $USER_EXPERIMENT_DIR/export \
                                --data_type int8 \
                                --cal_json_file $USER_EXPERIMENT_DIR/export/isbi_cal_qat.json \
                                --cal_cache_file $USER_EXPERIMENT_DIR/export/isbi_cal_qat.bin \
                                --max_batch_size 3 \
                                --batch_size 1

### D. Evaluate a QAT trained model using the exported TensorRT engine <a class="anchor" id="head-12-4"></a>
This section evaluates a QAT enabled pruned retrained model using the TensorRT int8 engine that was exported in [Section C](#head-12-3). Please note that there maybe a slight difference (~0.1-0.5%) in the MIOU from [Section B](#head-12-2), owing to some differences in the implementation of quantization in TensorRT.

*Note: The TensorRT evaluator might be slightly slower than the TAO evaluator here, because the evaluation dataloader is pinned to the CPU to avoid any clashes between TensorRT and TAO instances in the GPU. Please note that this tool was not intended and has not been developed for profiling the model. It is just a means to qualitatively analyse the model.*

*Please use native TensorRT or DeepStream for the most optimized inferences.*

In [ ]:
# Running inference on test dataset through tao deploy
!tao deploy unet evaluate --gpu_index $GPU_INDEX -e $SPECS_DIR/unet_train_resnet_unet_isbi_retrain_qat.txt \
                          -m $USER_EXPERIMENT_DIR/export/int8.isbi.retrained_qat.engine \
                          -r $USER_EXPERIMENT_DIR/isbi_experiment_pruned_qat/

### E. Inference using QAT engine <a class="anchor" id="head-12-5"></a>
Run inference and visualize detections on test images, using the exported TensorRT engine from [Section C](#head-12-3).

In [ ]:
# Running inference on test dataset through tao deploy
!tao deploy unet inference --gpu_index $GPU_INDEX -e $SPECS_DIR/unet_train_resnet_unet_isbi_retrain_qat.txt \
                           -m $USER_EXPERIMENT_DIR/export/int8.isbi.retrained_qat.engine \
                           -r $USER_EXPERIMENT_DIR/isbi_experiment_pruned_qat/

In [ ]:
# Visualizing the sample images.
OUTPUT_PATH = 'vis_overlay' # relative path from $USER_EXPERIMENT_DIR.
COLS = 2 # number of columns in the visualizer grid.
IMAGES = 4 # number of images to visualize.

visualize_images(OUTPUT_PATH, "isbi_experiment_pruned_qat", num_cols=COLS, num_images=IMAGES)